In [10]:
import sys
sys.path.append("../code")
import pandas as pd
import random
from glob import glob
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np

In [2]:
import blotto_utils as bu
from agent import Agent
import agent_utils as au

In [3]:
import scipy.optimize

In [4]:
%load_ext autoreload
%autoreload 2

In [5]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

# 538 Data

In [6]:
weights = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

data_dir = '/Users/Alex/Documents/Alex/Yale/Fall 2019/CPSC_490/blotto/data/'

fivethirtyeight_data_dir = data_dir + 'fivethirtyeight/'

sorted(glob(fivethirtyeight_data_dir + '*'))

temp_df = pd.read_csv(glob(fivethirtyeight_data_dir + '*')[0])

new_column_names = list(map((lambda x : '_'.join(x.split(' ')).lower()), list(temp_df.columns)))
new_column_names[-1] = 'reason'

temp_df.rename(dict(zip(list(temp_df.columns), new_column_names)), axis=1, inplace=True)

temp_df['total_troops'] = temp_df['castle_1'] + \
                          temp_df['castle_2'] + \
                          temp_df['castle_3'] + \
                          temp_df['castle_4'] + \
                          temp_df['castle_5'] + \
                          temp_df['castle_6'] + \
                          temp_df['castle_7'] + \
                          temp_df['castle_8'] + \
                          temp_df['castle_9'] + \
                          temp_df['castle_10']

full_strats_df = temp_df[temp_df.total_troops == 100].copy()

full_strats_list = full_strats_df.to_dict('records')

simp_strats_list = [[int(x[1]) for x in list(y.items())[:10]] for y in full_strats_list]

# GA

In [7]:
random.seed(0)

In [8]:
temp_noise = bu.generate_noise_arrays(1)

In [11]:
def evolve(a_strats, b_strats, generations, noise):
    for _ in range(generations):
        a_results, b_results = bu.eval_strats(a_strats, b_strats, weights)

        a_fittest = sorted(a_results, key=(lambda x: (x[1], -(x[0][-1]))), reverse=True)
        b_fittest = sorted(b_results, key=(lambda x: (x[1], -(x[0][-1]))), reverse=True)

        a_next_gen = []
        b_next_gen = []
        for i in range(14):
            for j in range(i+1, 15):
                a_next_gen.append(bu.mutate_noise(bu.make_integer(bu.cross([a_fittest[i][0], a_fittest[j][0]]), seed=(i+j+i*j)), temp_noise, seed=(i+j)))
                b_next_gen.append(bu.mutate_noise(bu.make_integer(bu.cross([b_fittest[i][0], b_fittest[j][0]]), seed=(i+j+i*j)), temp_noise, seed=(i+j)))

        a_strats = a_next_gen
        b_strats = b_next_gen
        
    return a_fittest, b_fittest, a_strats, b_strats

In [12]:
a_strat_init = random.sample(simp_strats_list, 100)
b_strat_init = random.sample(simp_strats_list, 100)

a_strats = a_strat_init
b_strats = b_strat_init

In [13]:
temp_a_strats, temp_b_strats = a_strats, b_strats

In [14]:
best_a, best_b = [], []

In [15]:
for i in range(200):
#     print(i)
    temp_a_fittest, temp_b_fittest, temp_a_strats, temp_b_strats = evolve(temp_a_strats, temp_b_strats, 1, temp_noise)
    best_a.append(temp_a_fittest[0][0])
    best_b.append(temp_b_fittest[0][0])
#     print('\n'.join(map(str,temp_a_fittest[:5])))

In [16]:
len(best_a), len(best_b)

(200, 200)

## Agent

In [17]:
agent_evol_a = Agent(best_a)
agent_evol_b = Agent(best_b)

In [18]:
print(0.5 + au.play_game(agent_evol_a, simp_strats_list, weights, weights, trials=len(simp_strats_list)) / (2*len(simp_strats_list)))
print(0.5 + au.play_game(agent_evol_b, simp_strats_list, weights, weights, trials=len(simp_strats_list)) / (2*len(simp_strats_list)))

0.30011074197120713
0.22591362126245845


# Linear Programming Testing (Nash)

input are two sets of strategies -> probabilities associated with each for Nash

In [19]:
probs_a, probs_b = bu.blotto_nash(a_strat_init, b_strat_init)

In [20]:
sum(probs_a), sum(probs_b)

(1.0000009999999995, 1.8453710000000005)

In [21]:
probs_a_norm = bu.normalize_probabilities(probs_a)
probs_b_norm = bu.normalize_probabilities(probs_b)

In [22]:
print(sum(probs_a_norm))
print(sum(probs_b_norm))

0.9999999999999999
1.0


In [23]:
agent_a = Agent(a_strat_init, list(probs_a_norm))
agent_b = Agent(b_strat_init, list(probs_b_norm))

In [24]:
au.play_game(agent_a, agent_b, weights, weights, trials=1000) / 1000 # b is the better agent on average

-0.15

In [25]:
print(0.5 + au.play_game(agent_a, simp_strats_list, weights, weights, trials=len(simp_strats_list)) / (2*len(simp_strats_list)))
print(0.5 + au.play_game(agent_b, simp_strats_list, weights, weights, trials=len(simp_strats_list)) / (2*len(simp_strats_list)))

0.6057585825027686
0.5481727574750831


## linear programming evolutionary strategies

In [26]:
probs_evol_a, probs_evol_b = bu.blotto_nash(best_a, best_b)

In [27]:
sum(probs_evol_a), sum(probs_evol_b)

(1.0000010000000006, 2.8886500000000015)

In [28]:
probs_evol_a_norm = bu.normalize_probabilities(probs_evol_a)
probs_evol_b_norm = bu.normalize_probabilities(probs_evol_b)

In [29]:
sum(probs_evol_a_norm), sum(probs_evol_b_norm)

(0.9999999999999998, 0.9999999999999998)

In [30]:
agent_evol_lin_a = Agent(best_a, list(probs_evol_a_norm))
agent_evol_lin_b = Agent(best_b, list(probs_evol_b_norm))

In [37]:
# random sample of training set
print(0.5 + au.play_game(agent_evol_lin_a, simp_strats_list, trials=len(simp_strats_list)) / (2*len(simp_strats_list)))
print(0.5 + au.play_game(agent_evol_lin_b, simp_strats_list, trials=len(simp_strats_list)) / (2*len(simp_strats_list)))

0.40420819490586934
0.43853820598006643


## Agents against each other

In [32]:
trials = 10000

In [33]:
print(0.5 + au.play_game(agent_evol_lin_a, agent_a, trials=trials)/(2*trials))
print(0.5 + au.play_game(agent_evol_lin_b, agent_b, trials=trials)/(2*trials))

0.35619999999999996
0.594


In [34]:
print(0.5 + au.play_game(agent_evol_lin_a, agent_evol_a, trials=trials)/(2*trials))
print(0.5 + au.play_game(agent_evol_lin_b, agent_evol_b, trials=trials)/(2*trials))

0.6439
0.6842


In [35]:
print(0.5 + au.play_game(agent_evol_a, agent_a, trials=trials)/(2*trials))
print(0.5 + au.play_game(agent_evol_b, agent_b, trials=trials)/(2*trials))

0.3025
0.3109
